#### Notebook demonstrating GET, SET, DEL, KEYS, and use of namespaces in Redis

In [1]:
import json
import pandas as pd
import random
import re
import redis
import uuid 

In [2]:
with open('redis.json', 'r') as f:
    config = json.load(f)
print(config)

{'__info__': 'Details needed to connect a Redis instance', 'host': 'app-test', 'port': 6378, 'namespace': 'rlmol'}


In [3]:
# Connect to Redis and print out keys under the namespace specified in redis.json

rconn = redis.Redis(host=config["host"], port=config["port"])
rconn.keys("%s:*" % config["namespace"])

[]

In [4]:
# Generate and save a bunch of random keys/values 

for _ in range(1000):
    key = uuid.uuid1() 
    value = random.uniform(0.0, 1.0)
    
    rconn.set("%s:%s" % (config["namespace"], key), value)

In [5]:
# As an example of simple analysis, pull out all keys with numerical values, sort them, and output top 10

numerical_keys = {}
for key in rconn.keys("%s:*" % config["namespace"]):
    try:
        value = float(rconn.get(key))
    except:
        continue
    
    # Convert byte string to regular string & remove namespace to keep the output concise
    key_clean = re.sub("%s:" % config["namespace"], "", key.decode('utf-8'))
    numerical_keys[key_clean] = value
    
all_values = pd.DataFrame({"value": list(numerical_keys.values())}, index=numerical_keys.keys())
display(all_values.sort_values("value", ascending=False)[:10])

,value
556fddc4-ecc4-11ea-9eac-2000110dfec0,0.997808
557983c4-ecc4-11ea-9eac-2000110dfec0,0.997604
5571533e-ecc4-11ea-9eac-2000110dfec0,0.997491
556dad1a-ecc4-11ea-9eac-2000110dfec0,0.995721
55700baa-ecc4-11ea-9eac-2000110dfec0,0.994555
5578a152-ecc4-11ea-9eac-2000110dfec0,0.994448
55798e00-ecc4-11ea-9eac-2000110dfec0,0.993108
5570d4c2-ecc4-11ea-9eac-2000110dfec0,0.993016
5564eedc-ecc4-11ea-9eac-2000110dfec0,0.992657
5573c696-ecc4-11ea-9eac-2000110dfec0,0.991293


In [6]:
# Clean up -- drop all keys in the namespace
for key in rconn.keys("%s:*" % config["namespace"]):
    rconn.delete(key)
    
print(rconn.keys("%s:*" % config["namespace"]))

[]
